In [ ]:
from main import *

# toy example

train_sub = [1]
train_x0, train_y0 = make_train_dataset(train_sub, 0)
score_model0 = train_scorenet(train_x0, train_y0)

/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
  0%|                                                                                                                                                                                                                              | 0/3000 [00:00<?, ?it/s]/home/junyeobe/augment/train_scorenet.py:184: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, device="cuda")
Score model Average Loss: 1002.109450, lr : 0.01:   6%|█████████▍                                                                                                       

In [ ]:
batch_size = 1
samples0 = sampling(score_model0, sample_batch_size=batch_size)

generated_signal0 = return_to_signal(samples0)

In [ ]:
import matplotlib.pyplot as plt


for sample in samples0:
    plt.plot(sample[0][0].cpu())
    plt.show()

for signal in generated_signal0[0][0]:
    plt.plot(signal)
    plt.show()


for x in train_x0:
    plt.plot(x[0])
    plt.show()
    
